# LLM Parameter-Efficient Fine-Tuning (PEFT) via HuggingFace Trainer APIs
Similar to last section [LLM Supervised Fine-Tuning (SFT)](../08.2_llm_sft/LLM_SFT.ipynb), in this section, we illustrate how to use [NVIDIA FLARE](https://nvidia.github.io/NVFlare) for Large Language Models (LLMs) PEFT task with [HuggingFace](https://huggingface.co/) Trainer APIs with [PEFT library](https://github.com/huggingface/peft).

We use the same model of the [Llama-3.2-1B model](https://huggingface.co/meta-llama/Llama-3.2-1B) to showcase the functionality of federated PEFT. For PEFT, we used LoRA method, other PEFT methods (e.g. p-tuning, prompt-tuning) can be easily adapted as well by modifying the configs following [PEFT](https://github.com/huggingface/peft) examples.

We conducted these experiments on a single 48GB RTX 6000 Ada GPU. 

To use Llama-3.2-1B model, please request access to the model here https://huggingface.co/meta-llama/Llama-3.2-1B and login with an access token using huggingface-cli.

## Setup
Git LFS is also necessary for downloads, please follow the steps in this [link](https://github.com/git-lfs/git-lfs/blob/main/INSTALLING.md).

Install required packages for training:

In [ ]:
%pip install -r requirements.txt

## Data Preparation
In this example, we use two datasets to illustrate the PEFT training.

We download and preprocess three data sets: [Dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k), and [Oasst1](https://huggingface.co/datasets/OpenAssistant/oasst1):

In [ ]:
! git clone https://huggingface.co/datasets/databricks/databricks-dolly-15k /tmp/nvflare/dataset/llm/dolly
! git clone https://huggingface.co/datasets/OpenAssistant/oasst1 /tmp/nvflare/dataset/llm/oasst1

In [ ]:
! python utils/preprocess_dolly.py --training_file /tmp/nvflare/dataset/llm/dolly/databricks-dolly-15k.jsonl --output_dir /tmp/nvflare/dataset/llm/dolly
! python utils/preprocess_oasst1.py --training_file /tmp/nvflare/dataset/llm/oasst1/data/train-00000-of-00001-b42a775f407cee45.parquet --validation_file /tmp/nvflare/dataset/llm/oasst1/data/validation-00000-of-00001-134b8fd0c89408b6.parquet --output_dir /tmp/nvflare/dataset/llm/oasst1

## Centralized Baseline
We run three centralized baselines as

In [ ]:
! python utils/hf_sft_peft.py --output_path /tmp/nvflare/workspace/llm/dolly_cen_peft --train_mode PEFT
! python utils/hf_sft_peft.py --data_path_train /tmp/nvflare/dataset/llm/oasst1/training.jsonl --data_path_valid /tmp/nvflare/dataset/llm/oasst1/validation.jsonl --output_path /tmp/nvflare/workspace/llm/oasst_cen_peft --train_mode PEFT

### Federated Training Results
We run the federated training on a single client using NVFlare Simulator via [JobAPI](https://nvflare.readthedocs.io/en/main/programming_guide/fed_job_api.html).

In [ ]:
! python peft_job.py --client_ids dolly oasst1 --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/all_fl_peft --job_dir /tmp/nvflare/workspace/jobs/llm_fl_peft --train_mode PEFT --threads 2 

The SFT curves are shown below.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/workspace/llm

With SFT and PEFT examples, now let's move on to the next section of [LLM Quantization](../08.4_llm_quantization/LLM_quantization.ipynb), where we will see how to make the message transmission more efficient.